In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('/Users/hzm/Desktop/YangTeng/code/venv/code/00.module')
import Proxy
import UA
import time

# = = = = = = = = = = = = = = = = = =

import pandas as pd
from selenium.webdriver import ChromeOptions

option = ChromeOptions()
option.add_experimental_option('useAutomationExtension', False)
option.add_argument('blink-settings=imagesEnabled=false')
option.add_argument('--disable-blink-features=AutomationControlled')
option.add_argument('--disable-dev-shm-usage')
option.add_argument('--disable-gpu')
option.add_experimental_option('excludeSwitches', ['enable-automation'])
option.add_argument('--headless')
option.add_argument('--no-sandbox')
option.add_argument('--proxy-server=' + Proxy.get_Proxy_Selenium())
option.add_argument('user-agent=' + UA.get_UA())
option.page_load_strategy = 'normal'

from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By

# = = = = = = = = = = = = = = = = = =

df_menu = pd.read_excel('./part.xlsx', header=0).fillna('')
list_info = df_menu['Info'].to_list()

print('总数量：' + str(len(list_info)))
print()

df = pd.DataFrame()

# = = = = = = = = = = = = = = = = = =

work = Queue()
for info in list_info:
    work.put_nowait(info)

def crawler():
    global df
    
    while not work.empty():
        info = work.get_nowait()
        
        # = = = = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(11):
            if info == None or info == '':
                status = 'none'
                break
            
            try:
                browser = Chrome(service=Service('../../../chromedriver'), options=option)
                browser.maximize_window()
                
                # = = = = = = = = = = = = = = = = = =

                browser.get(info)
                
                time.sleep(3)
                
                # = = = = = = = = = = = = = = = = = =
                
                wait = WebDriverWait(browser, 60)
                wait.until(EC.presence_of_element_located((By.XPATH, '//section[contains(@aria-label, "Warranty Information")]')))
                list_row = browser.find_elements(by=By.XPATH, value='//tbody[contains(@class, "listing-inner") and (contains(@class, "altrow-b-1") or contains(@class, "altrow-b-0"))]')

                # = = = = = = = = = = = = = = = = = =
                
                if len(list_row) == 0:
                    status = 'none'
                    break
                
                # = = = = = = = = = = = = = = = = = =
                
                list_kit_no = [i+1 for i in range(len(list_row))]
                list_kit_quantity = []
                list_kit_note = []
                list_kit_manufacturer = []
                list_kit_part_number = []
                list_kit_info = []

                for row in list_row:
                    list_td = row.find_elements(by=By.XPATH, value='./tr/td')
                    list_kit_quantity.append(list_td[0].text)
                    list_kit_note.append(list_td[1].text)
                    list_kit_manufacturer.append(list_td[2].find_element(by=By.XPATH, value='./descendant::span[contains(@class, "listing-final-manufacturer")]').text)
                    list_kit_part_number.append(list_td[2].find_element(by=By.XPATH, value='./descendant::span[contains(@class, "listing-final-partnumber")]').text)
                    list_kit_info.append(list_td[2].find_element(by=By.XPATH, value='./descendant::a[contains(@class, "ra-btn") and contains(@class, "ra-btn-moreinfo")]').get_attribute('href'))

                # = = = = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame({'status': 'ok',
                                        'Info': info,
                                        'Kit_No.': list_kit_no,
                                        'Kit_Quantity': list_kit_quantity,
                                        'Kit_Note': list_kit_note,
                                        'Kit_Manufacturer': list_kit_manufacturer,
                                        'Kit_Part_Number': list_kit_part_number,
                                        'Kit_Info': list_kit_info})

                browser.quit()
                
                status = 'ok'
                
                break
                
            except:
                time.sleep(3)
                continue
                
        # = = = = = = = = = = = = = = = = = =

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Info': info}])
        elif status == 'none':
            df_temp = pd.DataFrame([{'status': 'none',
                                     'Info': info}])

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        if info != None and info != '':
            print(info +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()))
        
# = = = = = = = = = = = = = = = = = =

list_task = []
for _ in range(25):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print()
print('输出中...')
df.to_excel('./kit.xlsx', index=False)
print()
print('搞定！')

总数量：19

https://www.rockauto.com/en/moreinfo.php?pk=40817  <->  [none] - 剩余数量：0
https://www.rockauto.com/en/moreinfo.php?pk=3771312  <->  [none] - 剩余数量：0
https://www.rockauto.com/en/moreinfo.php?pk=950845  <->  [none] - 剩余数量：0
https://www.rockauto.com/en/moreinfo.php?pk=40838  <->  [none] - 剩余数量：0
https://www.rockauto.com/en/moreinfo.php?pk=40826  <->  [none] - 剩余数量：0
https://www.rockauto.com/en/moreinfo.php?pk=40825  <->  [none] - 剩余数量：0
https://www.rockauto.com/en/moreinfo.php?pk=40816  <->  [none] - 剩余数量：0
https://www.rockauto.com/en/moreinfo.php?pk=1005840  <->  [none] - 剩余数量：0
https://www.rockauto.com/en/moreinfo.php?pk=40843  <->  [none] - 剩余数量：0
https://www.rockauto.com/en/moreinfo.php?pk=1001943  <->  [none] - 剩余数量：0
https://www.rockauto.com/en/moreinfo.php?pk=40829  <->  [none] - 剩余数量：0
https://www.rockauto.com/en/moreinfo.php?pk=40840  <->  [none] - 剩余数量：0
https://www.rockauto.com/en/moreinfo.php?pk=40835  <->  [none] - 剩余数量：0
https://www.rockauto.com/en/moreinfo.php?pk=40828